In [8]:
import pandas as pd
import numpy as np
data=pd.read_csv('지이인짜최종리뷰데이터.csv')
data.head()

,관광지,전체내용
0,경복궁,대한민국의 역사 : 대한민국의 역사가 잠들어 있는 곳. 서울을 방문했다면 꼭 방문해...
1,경복궁,국민이 공감하는 장소 : 경복궁은 국민들이 자주 찾는곳으로 작성자는 주말에 자주 가...
2,경복궁,산책하기 좋은 경복궁 : 날씨 좋은 날 종종 산책하러 경복궁에 가는데 마음이 편온해...
3,경복궁,Good : Goooooood 다 좋습니다 다음에 또 오고 싶네요 근처 관광지도많고...
4,경복궁,가족단위로 방문하기 좋은곳 : 요새 더더욱 코로나로 인해 사람 방문이 적음. 두자녀...


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

pres=['https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=72054&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=sympathyScore&page=',
     'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=72054&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=lowest&page=']

review=[]
rate=[]
target=[]

for pre in pres:
    for i in range(1,150):
        url=pre+str(i)
        res=requests.get(url)
        soup=BeautifulSoup(res.content,'html.parser')
    
        id_list=[]
        id_pre='_filtered_ment_'
    
        for i in range(10):
            id_list.append(id_pre+str(i))
        
        for id in id_list:
            review.append(soup.find('span',{'id':id}).get_text().strip())
            
        rate_list=[]
        rate_list=(soup.select('div.star_score > em'))
        
        for i in range(10):
            r=int(re.sub('<.+?>','',str(rate_list[i])))
            rate.append(r)
            if r >= 8:
                target.append(1)
            elif r<=4:
                target.append(0)
            else:
                target.append(-1)
                
df=pd.DataFrame({'review':review,'rate':rate,'target':target})

In [3]:
df

,review,rate,target
0,돈보다 영화가 아까운 사례,10,1
1,히어로 영화중 가장 최고는 바로 배트맨시리즈... 더이상 크리스찬베일의 배트맨을 ...,10,1
2,전작에 비해 약간 부족한면도 잇지만 히어로시리즈 중의 명작임은 분명하다,10,1
3,영화의예술로 시작해(비긴즈) 영화의상징이되어(다크나이트) 영화의전설(라이즈)로 끝나다,10,1
4,이 영화가 10점이 아니라면 10점짜리 영화는 없다.,10,1
...,...,...,...
2975,난 왜 베트맨 스리즈가 재미가 없지?? 이영화도 언제부터 재밌어질까 기다렸는데 끝...,5,-1
2976,"진짜 베인이랑 배트맨 때문에 볼려다가 앤 해서웨이 나오고 나서 바로 끔 , 이쁜척 ...",5,-1
2977,조커가 너무 쎘어...,5,-1
2978,나만 재미없고 지루한건가..두번을 봣지만 두번다 졸리는..,5,-1


In [4]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(review,target,test_size=0.2,random_state=0)

In [5]:
# 텍스트 데이터 벡터화

from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
okt=Okt()

tfv=TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2),min_df=3,max_df=0.9)
tfv.fit(train_x) # 모델 적합

tfv_train_x=tfv.transform(train_x)
tfv_train_x

C:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


<2384x3676 sparse matrix of type '<class 'numpy.float64'>'
	with 42983 stored elements in Compressed Sparse Row format>

In [6]:
# 로지스틱 회귀모형으로 모델 적합

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

clf=LogisticRegression(random_state=0)
params={'C':[1,3,5,7,9]}
grid_cv=GridSearchCV(clf,param_grid=params,cv=4,scoring='accuracy',verbose=1)
grid_cv.fit(tfv_train_x,train_y) # 모델 적합

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    3.4s finished


GridSearchCV(cv=4, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [1, 3, 5, 7, 9]}, scoring='accuracy', verbose=1)

In [7]:
# 테스트 데이터를 통한 예측 정확도 산출
tfv_test_x=tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x,test_y)

0.8741610738255033

In [18]:
reviews=list(data.전체내용)

results=[]
for i in range(len(reviews)):
    review=reviews[i]
    my_review=tfv.transform([review])
    if (grid_cv.best_estimator_.predict(my_review)):
#         print('긍정리뷰')
        results.append('긍정리뷰')
    else:
        results.append('부정리뷰')

In [19]:
df_review=pd.DataFrame({'리뷰':reviews,'평가':results})
df_review

,리뷰,평가
0,대한민국의 역사 : 대한민국의 역사가 잠들어 있는 곳. 서울을 방문했다면 꼭 방문해...,긍정리뷰
1,국민이 공감하는 장소 : 경복궁은 국민들이 자주 찾는곳으로 작성자는 주말에 자주 가...,부정리뷰
2,산책하기 좋은 경복궁 : 날씨 좋은 날 종종 산책하러 경복궁에 가는데 마음이 편온해...,긍정리뷰
3,Good : Goooooood 다 좋습니다 다음에 또 오고 싶네요 근처 관광지도많고...,긍정리뷰
4,가족단위로 방문하기 좋은곳 : 요새 더더욱 코로나로 인해 사람 방문이 적음. 두자녀...,긍정리뷰
...,...,...
5231,저는 녹차밭을 별로 기대하고 가지 않았습니다.. 처음에는 안 들리고 지나가려고 했었...,부정리뷰
5232,보성 녹차밭,긍정리뷰
5233,참 멋지고 이쁘네요말로만듣고 이번에 가봤는데 좋았어요또 가보고싶었고요추천해요혼자갔는...,부정리뷰
5234,비오는 겨울 풍경도 아름다운 다원...,부정리뷰


In [11]:
reviews[0]

'대한민국의 역사 : 대한민국의 역사가 잠들어 있는 곳. 서울을 방문했다면 꼭 방문해야 되는 곳. 경복궁은 우리의 역사다. 넓은 경복궁을 산책할 수 있는 것은 언제나 행복이다.'

In [15]:
my_review=tfv.transform([review[0]])